In [1]:
import time
import warnings
from datetime import datetime

import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import (explained_variance_score, mean_squared_error,
                             r2_score)
from sklearn.model_selection import train_test_split

from preprocessing.dataprep import (data_preparation, encode_train_test,
                                    filter_tce_data)
from preprocessing.text import fixColumnName

warnings.filterwarnings('ignore')

In [2]:
start_time = time.time()
start_date = datetime.now()
print(f'Starting evaluation at {start_date.strftime("%d/%m/%Y %H:%M:%S")}')
print()
print('Loading data...')

data_loading_time = time.time()
data = pd.read_csv('../database/dadosTCE.csv',
                   low_memory=False, encoding='utf-8')
print(f'Data loading time: {(time.time() - data_loading_time):.2f}s')
print()

Starting evaluation at 21/06/2021 13:15:08

Loading data...
Data loading time: 17.00s



In [3]:
data_prep_time = time.time()
print('Preprocessing data...')

# data = data.sample(1000,).reset_index(drop=True)
data.columns = list(map(fixColumnName, data.columns))
data, _ = filter_tce_data(data, '../database/norel.xlsx')

categorical_columns = [
    # 'exercicio_do_orcamento_ano',
    # 'empenho_sequencial_empenho',
    'orgao',
    'orgao_sucessor_atual',
    'tipo_administracao_nome',
    'tipo_poder_nome',
    # 'classificacao_orcamentaria_descricao',
    'funcao',
    'subfuncao',
    'programa',
    'acao',
    'grupo_despesa',
    'elemento_despesa',
    # 'natureza_despesa_cod',
    # 'natureza_despesa_nome',
    # 'formalidade_nome',
    'modalidade_licitacao_nome',
    # 'fonte_recurso_cod',
    'fonte_recurso_nome',
    'beneficiario_cnpj',
    'beneficiario_cpf',
    'beneficiario_cpf/cnpj',
    'periodo',
    'empenho_numero_do_processo',
    # 'empenho_sequencial_empenho.1',
]

text_columns = [
    # 'beneficiario_nome',
    'empenho_historico',
]

numerical_columns = [
    # 'valor_empenhado',
    # 'valor_anulacao_empenho',
    # # 'valor_estorno_anulacao_empenho',
    # 'valor_cancelamento_empenho',
    # # 'valor_anulacao_cancelamento_empenho',
    # 'valor_saldo_do_empenho',
    # 'valor_liquidacao_empenho',
    # 'valor_anulacao_liquidacao_empenho',
    # 'valor_saldo_liquidado',
    # 'valor_ordem_de_pagamento',
    # 'valor_guia_recolhimento',
    # 'valor_anulacao_ordem_de_pagamento',
    # 'valor_estorno_anulacao_o._pagamento',
    # 'valor_estorno_guia_recolhimento',
    # 'valor_saldo_pago',
    # 'valor_saldo_a_pagar',
    # 'valor_a_liquidar',
    # 'valor_a_pagar_liquidado'
]

target = data.valor_empenhado

data = data.loc[:, (*categorical_columns,
                    *text_columns,
                    *numerical_columns,)]

data, categorical_columns, numerical_columns = data_preparation(
    data,
    target,
    categorical_columns=categorical_columns,
    numerical_columns=numerical_columns,
    text_columns=text_columns,)

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.3, random_state=15)

X_train, X_test = encode_train_test(
    X_train, X_test, numerical_columns, categorical_columns, text_columns, tfidf=True,)

print(f'Data preparation time: {(time.time() - data_prep_time):.2f}s')
print(f'Training shape: {X_train.shape}')
print(f'Test shape: {X_test.shape}')
print()

Preprocessing data...
Data preparation time: 9.22s
Training shape: (465, 2956)
Test shape: (200, 2956)



In [4]:
rf_time = time.time()
print('Training RandomForest...')
reg = RandomForestRegressor(n_estimators=1000, random_state=15, n_jobs=20)
reg.fit(X_train, y_train)

print(f'RF training time: {(time.time() - rf_time):.2f}s')
print()

rf_time = time.time()
y_pred = reg.predict(X_test)

print(f'RF evaluation time: {(time.time() - rf_time):.2f}s')
print()

print(f'EVS: {explained_variance_score(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'R^2: {r2_score(y_test, y_pred)}')
print()

Training RandomForest...


C:\Users\jefma\Miniconda3\envs\tce\lib\site-packages\sklearn\utils\validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


RF training time: 229.30s



C:\Users\jefma\Miniconda3\envs\tce\lib\site-packages\sklearn\utils\validation.py:572: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


RF training time: 0.91s

EVS: 0.01797110128649171
MSE: 6218041957327.405
R^2: 0.014558297369730355



In [5]:
finish_date = datetime.now()
print(f'Finishing evaluation at {finish_date.strftime("%d/%m/%Y %H:%M:%S")}')

Finishing evaluation at 21/06/2021 13:19:25
